## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")
os.environ["DASHSCOPE_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")

### 如何使用通义千问
### How to use Tongyi
****

### LLM资源获取：
- 打开 https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key?spm=a2c4g.2399481.0.0
- 按说明操作
- 获得API-KEY
- 安装包
- 按照教程可以接入langchain
- 注意：由于模型调教和参数关系，部分Agent type 千问是不支持的！类似create_react_agent这种比较简单的支持度会比较好！
- agent type见官网 https://python.langchain.com/docs/modules/agents/agent_types/

### LLM Resource Acquisition:
- Open https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key?spm=a2c4g.2399481.0.0
- Follow the instructions
- Obtain API-KEY
- Installation package
- You can integrate langchain according to the tutorial
- Note: Due to model tuning and parameter relationships, some Agent types such as Q&A are not supported! Simpler types like create_react_agent have better support!
- Agent types can be found on the official website https://python.langchain.com/docs/modules/agents/agent_types/
<hr>

In [4]:
# Install the package
! pip install --upgrade --quiet  dashscope  -i https://mirrors.aliyun.com/pypi/simple/


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.llms import Tongyi

In [3]:
Tongyi().invoke("What NFL team won the Super Bowl in the year Justin Bieber was born?")

'Justin Bieber was born on March 1, 1994. The Super Bowl that took place in the early part of 1994 (January 29, 1994) was Super Bowl XXVIII. In that game, the Dallas Cowboys defeated the Buffalo Bills with a score of 30-13. Therefore, the NFL team that won the Super Bowl in the year Justin Bieber was born is the Dallas Cowboys.'

### 使用chain
### Use the chain
<hr>

In [4]:
from langchain_core.prompts import PromptTemplate
llm = Tongyi()
template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

chain.invoke({"question": question})

"Sure, let's break it down step by step:\n\n1. First, we need to determine the year Justin Bieber was born.\n   - Justin Bieber was born on March 1, 1994.\n\n2. Now that we know the year, we need to find out which Super Bowl took place in 1994.\n   - The Super Bowl played in January of each year is for the previous year's NFL season. So, Super Bowl XXVIII (28) was played on January 30, 1994.\n\n3. Finally, we need to identify the winner of Super Bowl XXVIII.\n   - In Super Bowl XXVIII, the Dallas Cowboys defeated the Buffalo Bills with a score of 30-13.\n\nTherefore, the NFL team that won the Super Bowl in the year Justin Bieber was born (1994) was the Dallas Cowboys."

### 使用Agent
### Using Agents
<hr>

In [5]:

llm = Tongyi()

In [6]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [8]:
tools = [get_word_length]

In [7]:
from langchain import hub
# Get the prompt to use - you can modify this!
#prompt = hub.pull("hwchase17/react")
#create_json_chat_agent
prompt = hub.pull("hwchase17/react-chat-json")
#create_structured_chat_agent
#prompt = hub.pull("hwchase17/structured-chat-agent")

In [9]:
from langchain.agents import AgentExecutor, create_react_agent,create_structured_chat_agent,create_json_chat_agent
# Construct a agent
#agent = create_structured_chat_agent(llm, tools, prompt)
#agent = create_react_agent(llm, tools, prompt)
agent = create_json_chat_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [10]:
agent_executor.invoke({"input": "LangChain这个单词有几个字母?"})



> Entering new AgentExecutor chain...
```json
{
    "action": "get_word_length",
    "action_input": "LangChain"
}
```9```json
{
    "action": "Final Answer",
    "action_input": "LangChain这个单词有9个字母。"
}
```

> Finished chain.


{'input': 'LangChain这个单词有几个字母?', 'output': 'LangChain这个单词有9个字母。'}